##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 分散ストラテジーを使ってモデルを保存して読み込む

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> ノートブックをダウンロード</a></td>
</table>

## 概要

トレーニング中にモデルを保存して読み込むことは一般的な作業です。Keras モデルの保存と読み込みに使用する API には、高レベル API と低レベル API の 2 つがあります。このチュートリアルでは、`tf.distribute.Strategy` を使用してる場合に SavedModel API を使用する方法を実演しています。SavedModel とシリアル化に関する一般的な情報は、[SavedModel ガイド](../../guide/saved_model.ipynb)と[Keras モデルのシリアル化ガイド](../../guide/keras/save_and_serialize.ipynb)をご覧ください。では、簡単な例から始めましょう。 

依存関係をインポートします。

In [ ]:
import tensorflow_datasets as tfds

import tensorflow as tf
tfds.disable_progress_bar()


`tf.distribute.Strategy` を使ってデータとモデルを準備します。

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

モデルをトレーニングします。 

In [ ]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

## モデルを保存して読み込む

作業に使用する単純なモデルの準備ができたので、保存と読み込みの API を見てみましょう。利用できる API には次の 2 種類があります。

- 高レベルの Keras `model.save` と `tf.keras.models.load_model`
- 低レベルの `tf.saved_model.save` と `tf.saved_model.load`


### Keras API

次に、Keras API を使ってモデルを保存して読み込む例を示します。

In [ ]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)

`tf.distribute.Strategy` を使用せずにモデルを復元します。

In [ ]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

モデルを復元したら、`compile()` をもう一度呼び出すことなくそのトレーニングを続行できます。保存前にコンパイル済みであるからです。モデルは、TensorFlow の標準的な `SavedModel` プロと形式で保存されています。その他の詳細は、[`saved_model` 形式ガイド](../../guide/saved_model.ipynb)をご覧ください。

`tf.distribute.Strategy` を使用して、モデルを読み込んでトレーニングします。

In [ ]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

ご覧の通り、`tf.distribute.Strategy` を使って期待通りに読み込まれました。ここで使用されるストラテジーは、保存前に使用したストラテジーと同じものである必要はありません。 

### `tf.saved_model` API

では、低レベル API を見てみましょう。モデルの保存は Keras API に類似しています。

In [ ]:
model = get_model()  # get a fresh model
saved_model_path = "/tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

読み込みは `tf.saved_model.load()` で行えますが、より低いレベルにある API（したがって広範なユースケースのある API）であるため、Keras モデルを返しません。代わりに、推論を行うために使用できる関数を含むオブジェクトを返します。次に例を示します。

In [ ]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

読み込まれたオブジェクトには複数の関数が含まれ、それぞれにキーが関連付けられている可能性があります。`"serving_default"` は、保存された Keras モデルを使用した推論関数のデフォルトのキーです。この関数で推論を実行するには、次のようにします。 

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

また、分散方法で読み込んで推論を実行することもできます。

In [ ]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(
      predict_dataset)

  # Calling the function in a distributed manner
  for batch in dist_predict_dataset:
    another_strategy.run(inference_func,args=(batch,))

復元された関数の呼び出しは、保存されたモデル（predict）に対するフォワードパスです。読み込まれた関数をトレーニングし続ける場合はどうでしょうか。または読み込まれた関数をより大きなモデルに埋め込むには？一般的には、この読み込まれたオブジェクトを Keras レイヤーにラップして行うことができます。幸いにも、[TF Hub](https://www.tensorflow.org/hub) には、次に示すとおり、この目的に使用できる [hub.KerasLayer](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/keras_layer.py) が用意されています。

In [ ]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28, 28, 1), name='input_x')
  # Wrap what's loaded to a KerasLayer
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  model.fit(train_dataset, epochs=2)

ご覧の通り、`hub.KerasLayer` は `tf.saved_model.load()` から読み込まれた結果を、別のモデルの構築に使用できる Keras レイヤーにラップしています。学習を転送する場合に非常に便利な手法です。 

### どの API を使用すべきですか？

保存の場合は、Keras モデルを使用しているのであれば、Keras の `model.save()` API をほぼ必ず使用することが推奨されます。保存しているものが Keras モデルでなければ、低レベル API しか使用できません。

読み込みの場合は、読み込み API から得ようとしているものによって選択肢がきまs理ます。Keras モデルを使用できない（または使用を希望しない）のであれば、`tf.saved_model.load()` を使用し、そうでなければ、`tf.keras.models.load_model()` を使用します。Keras モデルを保存した場合にのみ Keras モデルを読み込めることに注意してください。

API を混在させることも可能です。`model.save` で Keras モデルを保存し、低レベルの `tf.saved_model.load` API を使用して、非 Keras モデルを読み込むことができます。 

In [ ]:
model = get_model()

# Saving the model using Keras's save() API
model.save(keras_model_path) 

another_strategy = tf.distribute.MirroredStrategy()
# Loading the model using lower level API
with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

### ローカルデバイスで保存または読み込む

クラウド TPU を使用するなど、リモートで実行中にローカルの IO デバイスに保存したり、そこから読み込んだりする場合、`experimental_io_device` オプションを使用して、IO デバイスを localhost に設定する必要があります。

In [ ]:
model = get_model()

# Saving the model to a path on localhost.
saved_model_path = "/tmp/tf_save"
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save(saved_model_path, options=save_options)

# Loading the model from a path on localhost.
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
  loaded = tf.keras.models.load_model(saved_model_path, options=load_options)

### 警告

特殊なケースは、入力が十分に定義されていない Keras モデルがある場合です。たとえば、Seeuqntial モデルは、入力形状（`Sequential([Dense(3), ...]`）を使用せずに作成できます。Subclassed モデルにも、初期化後は十分に定義された入力がありません。この場合、保存と読み込みの両方に低レベル API を使用する必要があります。そうしない場合はエラーが発生します。

モデルの入力が十分に定義されたものであるかを確認するには、`model.inputs` が `None` であるかどうかを確認します。`None` でなければ問題ありません。入力形状は、モデルが `.fit`、`.evaluate`、`.predict` で使用されている場合、またはモデルを呼び出す場合（`model(inputs)`）に自動的に定義されます。

次に例を示します。

In [ ]:
class SubclassedModel(tf.keras.Model):

  output_name = 'output_layer'

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
        5, dtype=tf.dtypes.float32, name=self.output_name)

  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()
# my_model.save(keras_model_path)  # ERROR! 
tf.saved_model.save(my_model, saved_model_path)